# Лабораторная работа 4: Латентное размещение Дирихле. Лабуткин Иван Алексеевич.

## Импорт библиотек и подготовка данных

In [98]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
import time

In [99]:
from LDA import LDA

In [100]:
categories = ['soc.religion.christian','comp.graphics', 'sci.med','rec.motorcycles']
X,y = fetch_20newsgroups(categories=categories,return_X_y=True)
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=2025)

## Применение реализованного алгоритма

In [101]:
lda=LDA(num_themes=4, alpha=1.5, beta=1.5)

In [102]:
start = time.time()
phi_train,theta_train=lda.fit_predict(train_text=X_train,max_steps=500)
end = time.time()
print("Время выполнения программы:",
      (end-start) * 10**3, "ms")

Время выполнения программы: 506249.66168403625 ms


In [103]:
y_pred_train=np.argmax(theta_train,axis=0)
pd.crosstab(y_pred_train,y_train)

col_0,0,1,2,3
row_0,,,,
0,8,5,51,464
1,7,468,64,3
2,442,5,18,10
3,3,5,340,7


In [104]:
def get_accuracy(matches_dict,y_pred,y_real):#Расчёт accuracy
    trues=0
    for ind_pred, ind_real in matches_dict.items():
        trues+=np.sum(np.isin(np.where(y_pred==ind_pred), np.where(y_real==ind_real)))
    return trues/len(y_real)

In [105]:
matches={
    0:3,
    1:1,
    2:0,
    3:2
}
print("Точность на тренировочных данных:",round(get_accuracy(matches,y_pred_train,y_train),3))

Точность на тренировочных данных: 0.902


In [109]:
start = time.time()
theta_test=lda.predict(X_test, max_steps=500)
end = time.time()
print("Время выполнения программы:",
      (end-start) * 10**3, "ms")


Время выполнения программы: 490.2322292327881 ms


In [110]:
y_pred_test=np.argmax(theta_test,axis=0)
pd.crosstab(y_test,y_pred_test)

col_0,0,1,2,3
row_0,,,,
0,19,15,78,12
1,21,66,15,13
2,33,26,9,53
3,80,22,7,6


In [111]:
print("Точность на тренировочных данных:",round(get_accuracy(matches,y_pred_test,y_test),3))

Точность на тренировочных данных: 0.583


## Сравнение с эталонным решением

In [112]:
from sklearn.decomposition import LatentDirichletAllocation


In [137]:
lda_etalon=LatentDirichletAllocation(n_components=4)
start = time.time()
X_t=lda.vectorizer.transform(X_train)
etalon_matrix=lda_etalon.fit_transform(X_t)
end = time.time()
print("Время выполнения программы:",
      (end-start) * 10**3, "ms")

Время выполнения программы: 5197.357177734375 ms


In [138]:
y_etalon_pred_train=np.argmax(etalon_matrix,axis=1)
pd.crosstab(y_etalon_pred_train,y_train)

col_0,0,1,2,3
row_0,,,,
0,146,352,247,64
1,193,118,198,28
2,113,9,8,3
3,8,4,20,389


In [139]:
matches_etalon={
    0:1,
    1:2,
    2:0,
    3:3
}
print("Точность на тренировочных данных:",round(get_accuracy(matches_etalon,y_etalon_pred_train,y_train),3))

Точность на тренировочных данных: 0.554


In [120]:
X_t_test=lda.vectorizer.transform(X_test)
etalon_matrix_test=lda_etalon.fit_transform(X_t_test)

In [121]:
y_etalon_pred_test=np.argmax(etalon_matrix_test,axis=1)
pd.crosstab(y_etalon_pred_test,y_test)

col_0,0,1,2,3
row_0,,,,
1,1,0,0,0
3,123,115,121,115


In [122]:
print("Точность на тренировочных данных:",round(get_accuracy(matches_etalon,y_etalon_pred_test,y_test),3))

Точность на тренировочных данных: 0.242


## Расчёт когерентностей тем

In [123]:
from LDA import get_coherence_score

In [127]:
get_coherence_score(lda.vectorizer, lda.phi, X_train, metric='u_mass')#По реализованному алгоритму

Когерентность тем (u_mass): -0.2462


In [140]:
etalon_phi=lda_etalon.components_/lda_etalon.components_.sum(axis=1, keepdims=True)

In [141]:
get_coherence_score(lda.vectorizer, etalon_phi, X_train, metric='u_mass')#По эталонному алгоритму

Когерентность тем (u_mass): -0.3968
